In [1]:
import os, sys
import ffmpeg
import json
import numpy as np
from ipywidgets import interact
from matplotlib import pyplot as plt
    
import sqlalchemy
from sqlalchemy import select

import cv2 as cv

from watcher import EventObservation, MotionEvent, TunneledConnection

In [2]:
VIDEO_BASE_DIR = '/Volumes/Video Captures'

tc = TunneledConnection().connect()
session = sqlalchemy.orm.Session(tc)

In [3]:
event_name = '20220406_101444_1709'

stmt = select(EventObservation).where(EventObservation.event_name == event_name)
event = session.execute(stmt).scalar()

stmt = select(MotionEvent).where(MotionEvent.event_name == event_name)
motions = session.execute(stmt).scalars().all()

In [4]:
video_file = event.file_path(alt_base_dir=VIDEO_BASE_DIR)
info = ffmpeg.probe(video_file)

video_info = next(stream for stream in info['streams'] if stream['codec_type'] == 'video')
width = int(video_info['width'])
height = int (video_info['height'])
num_frames = int(video_info['nb_frames'])

In [5]:
out, err = (
    ffmpeg
    .input(video_file)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run(capture_stdout=True)
);
video = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)

ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  

In [9]:
GREEN = (0, 255, 0)

for m in motions:
    (y, x, h, w) = m.box()
    f = m.frame

    p0 = (y,x)
    p1 = (y+h, x+w)

    cv.rectangle(video[f,:,:,:], p0, p1, GREEN, 2);

In [11]:
@interact(frame=(0, num_frames))
def show_frame(frame=0):
    plt.rcParams['figure.figsize'] = [15, 8]
    plt.imshow(video[frame,:,:,:])

interactive(children=(IntSlider(value=0, description='frame', max=579), Output()), _dom_classes=('widget-inter…